In [1]:
import ee
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from tqdm import tqdm

import subprocess
from joblib import Parallel, delayed
import multiprocessing

from sklearn.model_selection import StratifiedKFold, train_test_split, StratifiedGroupKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import xgboost as xgb
import lightgbm as lgb
import catboost as catb

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [4]:
path = './data/'

In [5]:
# Load files
train = pd.read_csv(f'{path}Train.csv')
test = pd.read_csv(f'{path}Test.csv')
sample_submission = pd.read_csv(f'{path}SampleSubmission.csv')

# Preview head of train
train.head()

,ID,Lat,Lon,Target
0,ID_SJ098E7S2SY9,34.162491,70.763668,0
1,ID_CWCD60FGJJYY,32.075695,48.492047,0
2,ID_R1XF70RMVGL3,14.542826,33.313483,1
3,ID_0ZBIDY0PEBVO,14.359480,33.284108,1
4,ID_C20R2C0AYIT0,14.419128,33.528450,0


In [6]:
lonlat_mapper = {
    '34-70': 'Afghanistan',
    '32-48': 'Iran',
    '14-33': 'Sudan',
}

In [7]:
train['location'] = train.apply(lambda x: f'{np.int8(x[1])}-{np.int8(x[2])}', axis=1).map(lonlat_mapper)
test['location'] = test.apply(lambda x: f'{np.int8(x[1])}-{np.int8(x[2])}', axis=1).map(lonlat_mapper)

In [8]:
train.head()

,ID,Lat,Lon,Target,location
0,ID_SJ098E7S2SY9,34.162491,70.763668,0,Afghanistan
1,ID_CWCD60FGJJYY,32.075695,48.492047,0,Iran
2,ID_R1XF70RMVGL3,14.542826,33.313483,1,Sudan
3,ID_0ZBIDY0PEBVO,14.359480,33.284108,1,Sudan
4,ID_C20R2C0AYIT0,14.419128,33.528450,0,Sudan


In [9]:
train.groupby(['location', 'Target']).count()

ID  Lat  Lon
location    Target               
Afghanistan 0       250  250  250
            1       250  250  250
Iran        0       250  250  250
            1       250  250  250
Sudan       0       250  250  250
            1       250  250  250

In [10]:
test.head()

,ID,Lat,Lon,location
0,ID_9ZLHTVF6NSU7,34.254835,70.348699,Afghanistan
1,ID_LNN7BFCVEZKA,32.009669,48.535526,Iran
2,ID_SOYSG7W04UH3,14.431884,33.399991,Sudan
3,ID_EAP7EXXV8ZDE,14.281866,33.441224,Sudan
4,ID_QPRX1TUQVGHU,14.399365,33.109566,Sudan


In [11]:
# Preview head of the sample submission
sample_submission.head()

,ID,Target
0,ID_9ZLHTVF6NSU7,
1,ID_LNN7BFCVEZKA,
2,ID_SOYSG7W04UH3,
3,ID_EAP7EXXV8ZDE,
4,ID_QPRX1TUQVGHU,


In [12]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B']

time_maps = {
    'Afghanistan': {
        'start': '2021-04-01',
        'end': '2022-04-30',
    },
    'Iran': {
        'start': '2019-07-01',
        'end': '2020-06-30',
    },
    'Sudan': {
        'start': '2019-07-01',
        'end': '2020-06-30',
    }
}

In [13]:
%%time
train_data = pd.read_csv('data/train_data.csv')
test_data = pd.read_csv('data/test_data.csv')

CPU times: user 14.5 ms, sys: 1.15 ms, total: 15.7 ms
Wall time: 17 ms


In [14]:
%%time
train_ts_data = pd.read_csv('data/train_ts_data.csv')
test_ts_data = pd.read_csv('data/test_ts_data.csv')

CPU times: user 209 ms, sys: 15.3 ms, total: 224 ms
Wall time: 226 ms


In [15]:
def agg_data(df):
    fncts = ['min']
    fs = [np.min]
    
    data = None
    for fn, f in zip(fncts, fs):
        if data is None:
            data = df.groupby('ID', as_index=False).agg(f).add_suffix(f'_{fn}').rename(columns={f'ID_{fn}': 'ID'})
        else:
            data = pd.merge(
                data, 
                df.groupby('ID', as_index=False).agg(f).add_suffix(f'_{fn}').rename(columns={f'ID_{fn}': 'ID'}),
                on='ID'
            )
        
    return data

In [16]:
def vegetation_indices(df):
    veg_df = pd.DataFrame()
    
    eps = 0
    
    veg_df['NDRE'] = (df['B8'] - df['B6'])/(df['B8'] + df['B6'] + eps)
    veg_df['MMSR'] = ((df['B8']/df['B4']) - 1) / ((df['B8']/df['B4'])**0.5 + 1)
    veg_df['NDWI'] = (df['B8A'] - df['B11'])/(df['B8A'] + df['B11'] + eps)
    veg_df['GNDVI'] = (df['B8'] - df['B3']) / (df['B8'] + df['B3'] + eps)
    veg_df['EVI2'] = 2.5 * (df['B8'] - df['B4']) / (df['B8'] + 2.4*df['B4'] + 1)
    veg_df['NGRDI'] = ((df['B3'] - df['B4'])/(df['B3'] + df['B4'] + eps))
    veg_df['MNDWI'] = ((df['B3'] - df['B11'])/(df['B3'] + df['B11'] + eps))
    veg_df['OSAVI'] = (df['B8'] - df['B4']) / (df['B8'] + df['B4'] + 0.35)
    veg_df['WDRVI'] = (0.1 * df['B8'] - df['B4'])/(0.1 * df['B8'] + df['B4'] + eps)
    veg_df['TGI'] =  df['B3'] - 0.39 * df['B4'] - 0.61 * df['B2']
    veg_df['GCVI'] = (df['B8'] / (df['B3'] + eps) ) - 1
    veg_df['RGVI'] = 1 - ((df['B1'] + df['B3']) / (df['B4'] + df['B5'] + df['B7'] + eps) )
    veg_df['MI'] = (df['B8A'] - df['B11']) / (df['B8A'] + df['B11'] + eps)
    veg_df['ARVI'] =  (df['B8'] - (2 * df['B4']) + df['B2']) / (df['B8'] + (2 * df['B4']) + df['B2'] + eps)
    veg_df['SIPI'] = (df['B8'] - df['B2']) / (df['B8'] - df['B4'] + eps)
    veg_df['RENDVI'] = (df['B6'] - df['B5']) / (df['B6'] + df['B5'] + eps) 
    veg_df['MRESR'] = (df['B6'] - df['B1']) / (df['B5'] - df['B1'] + eps)
    veg_df['RYI'] = df['B3'] / (df['B2'] + eps)
    veg_df['NDYI'] = (df['B3'] - df['B2']) / (df['B3'] + df['B2'] + eps)
    veg_df['DYI'] = df['B3'] - df['B2']
    veg_df['ACI'] = df['B8'] * (df['B4'] + df['B3'] + eps)
    veg_df['CVI'] = (df['B8'] / (df['B3'] + eps)) * (df['B4'] / (df['B3'] + eps))
    veg_df['AVI'] = (df['B8'] * (1 - df['B4']) * (df['B8'] - df['B4']))
    veg_df['SI'] = ((1 - df['B2']) * (1 - df['B3']) * (1 - df['B4']))
    veg_df['BSI'] = ((df['B11'] + df['B4']) - (df['B8'] + df['B2'])) / ((df['B11'] + df['B4']) + (df['B8'] + df['B2']) + eps)
    veg_df['SAVI'] = ((df['B8'] - df['B4'])/(df['B8'] + df['B4'] + 0.33)) * (1 + 0.33)
    veg_df['FIDET'] = df['B12'] / (df['B8A'] * df['B9']  + eps)
    veg_df['MTCI'] = (df['B6'] - df['B5'])/(df['B5'] - df['B4'] + eps)
    veg_df['NPCRI'] = (df['B4'] - df['B2']) / (df['B4'] + df['B2'] + eps)
    veg_df['S2REP'] = 705 + 35 * ((((df['B7'] + df['B4'])/2) - df['B5'])/(df['B6'] - df['B5'] + eps)) 
    veg_df['CCCI'] = ((df['B8'] - df['B5']) / (df['B8'] + df['B5'] + eps)) / ((df['B8'] - df['B4']) / (df['B8'] + df['B4'] + eps)) 
    veg_df['MCARI'] = ((df['B5'] - df['B4']) - 2 * (df['B5'] - df['B3'])) * (df['B5'] / (df['B4'] + eps))  
    veg_df['TCARI'] = 3 * ((df['B5'] - df['B4']) - 0.2 * (df['B5'] - df['B3']) * (df['B5'] / (df['B4'] + eps))) 
    veg_df['PVI'] = (df['B8'] - 0.3 * df['B4'] - 0.5) / ((1 + 0.3 * 2) ** 0.5) 
    veg_df['EVI'] = 2.5*(df['B8'] - df['B4']) / (df['B8'] + 6*df['B4'] - 7.5*df['B2'] + 1 + eps)
    veg_df['NDVI'] = ((df['B8'] - df['B4'])/(df['B8'] + df['B4'] + eps))
    veg_df['BAI'] = 1/((0.1 - df['B4']) ** 2 + (0.06 - df['B8']) ** 2)
    veg_df['MTVI2'] = list(1.5*(1.2 * (i - j) - 2.5 * (k - j))* ((2 * i + 1)**2 - (6 * i - 5 * k ** (0.5)) - 0.5)**(0.5) for i, j, k in zip(df['B8'], df['B3'], df['B4']))
    veg_df['NDSI'] = (df['B3'] - df['B11']) / (df['B3'] + df['B11'] + eps)
    veg_df['MRENDVI'] = (df['B6'] - df['B5']) / (df['B6'] + df['B5'] - 2 * df['B1'] + eps) 
    veg_df['NDVIre'] = (df['B8'] - df['B5'])/(df['B8'] + df['B5'] + eps)
    veg_df['CIre'] = ((df['B8'] / (df['B5'] + eps)) - 1)
    veg_df['NDMI'] = (df['B8'] - df['B11'])/(df['B8'] + df['B11'] + eps) 
    veg_df['TNDVI'] = ((df['B8'] - df['B4']) / (df['B8'] + df['B4'] + eps) + 0.5) ** 0.5
    veg_df['VDVI'] = (2 * df['B3'] - df['B4'] - df['B2']) / (2 * df['B3'] + df['B4']  + df['B2'] + eps)
    veg_df['NBR'] = (df['B8'] - df['B11']) / (df['B8'] + df['B11'] + eps)
    veg_df['TVI'] = (120 * (df['B6'] - df['B3']) - 200 * (df['B4'] - df['B3'])) / 2
    veg_df['EXG'] = 2 * df['B3'] - df['B4'] - df['B2']
    veg_df['PSRI'] = (df['B4'] - df['B2']) / (df['B6'] + eps)
    veg_df['RDVI'] = (df['B8'] - df['B4']) / (df['B8'] + df['B4'] + eps)**0.5

    veg_df['RATIO1'] = df['B1'] / (df['B3'] + eps)
    veg_df['RATIO2'] = df['B1'] / (df['B5'] + eps)
    veg_df['RATIO3'] = df['B11'] / (df['B12'] + eps)
    veg_df['RATIO4'] = df['B5'] / (df['B4'] + eps)
    veg_df['RATIO5'] = df['B7'] / (df['B5'] + eps)
    veg_df['RATIO6'] = df['B7'] / (df['B6'] + eps)
    veg_df['RATIO7'] = df['B8'] / (df['B4'] + eps)
    
    return veg_df

In [17]:
def add_veg_indices(_df):
    df = _df.copy()
    
    df = pd.concat([df, vegetation_indices(df)], axis=1)
    
    return df

In [18]:
train_data.head(2)

,ID,Lat,Lon,Target,location,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12,AOT,WVP,SCL,TCI_R,TCI_G,TCI_B
0,ID_SJ098E7S2SY9,34.162491,70.763668,0,Afghanistan,906.564103,1204.102564,1656.307692,2044.384615,2355.641026,2441.410256,2556.256410,2505.871795,2666.333333,2627.179487,3488.948718,3230.794872,293.897436,1220.205128,5.000000,208.641026,168.74359,122.487179
1,ID_CWCD60FGJJYY,32.075695,48.492047,0,Iran,509.023810,628.214286,907.428571,980.476190,1343.833333,2158.309524,2411.404762,2702.309524,2653.714286,3001.452381,2105.380952,1433.095238,207.833333,1250.833333,4.880952,100.023810,92.50000,64.404762


In [19]:
test_data.head(2)

,ID,Lat,Lon,location,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12,AOT,WVP,SCL,TCI_R,TCI_G,TCI_B
0,ID_9ZLHTVF6NSU7,34.254835,70.348699,Afghanistan,631.621622,881.202703,1312.513514,1409.081081,1848.824324,2574.040541,2857.500000,2914.486486,2942.256757,2994.608108,2765.608108,2266.500000,269.567568,1060.783784,4.702703,143.635135,134.040541,90.040541
1,ID_LNN7BFCVEZKA,32.009669,48.535526,Iran,976.666667,1393.642857,1914.380952,2407.476190,2633.666667,2824.809524,2989.738095,3185.166667,3120.904762,3217.047619,3631.547619,3163.190476,207.285714,1300.619048,5.000000,228.333333,194.785714,143.952381


In [20]:
train_ts_data.head(2)

,ID,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12,AOT,WVP,SCL,TCI_R,TCI_G,TCI_B
0,ID_SJ098E7S2SY9,739.0,1058.0,1490.0,1648.0,2005.0,2591.0,2750.0,2708.0,2848.0,2781.0,3153.0,2805.0,214.0,506.0,5.0,168.0,152.0,110.0
1,ID_SJ098E7S2SY9,778.0,966.0,1380.0,1574.0,1967.0,2487.0,2599.0,2644.0,2724.0,2689.0,3067.0,2651.0,246.0,664.0,5.0,159.0,141.0,101.0


In [21]:
test_ts_data.head(2)

,ID,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12,AOT,WVP,SCL,TCI_R,TCI_G,TCI_B
0,ID_9ZLHTVF6NSU7,421.0,307.0,606.0,466.0,950.0,2324.0,3006.0,3880.0,2926.0,3230.0,1589.0,1017.0,204.0,504.0,4.0,49.0,63.0,31.0
1,ID_9ZLHTVF6NSU7,425.0,319.0,616.0,478.0,960.0,2324.0,3000.0,3872.0,2922.0,3256.0,1592.0,1022.0,189.0,503.0,4.0,49.0,64.0,33.0


In [22]:
full_train = add_veg_indices(train_data)
full_test = add_veg_indices(test_data)

full_ts_train = add_veg_indices(train_ts_data)
full_ts_test = add_veg_indices(test_ts_data)

In [23]:
full_train.head()

,ID,Lat,Lon,Target,location,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12,AOT,WVP,SCL,TCI_R,TCI_G,TCI_B,NDRE,MMSR,NDWI,GNDVI,EVI2,NGRDI,MNDWI,OSAVI,WDRVI,TGI,GCVI,RGVI,MI,ARVI,SIPI,RENDVI,MRESR,RYI,NDYI,DYI,ACI,CVI,AVI,SI,BSI,SAVI,FIDET,MTCI,NPCRI,S2REP,CCCI,MCARI,TCARI,PVI,EVI,NDVI,BAI,MTVI2,NDSI,MRENDVI,NDVIre,CIre,NDMI,TNDVI,VDVI,NBR,TVI,EXG,PSRI,RDVI,RATIO1,RATIO2,RATIO3,RATIO4,RATIO5,RATIO6,RATIO7
0,ID_SJ098E7S2SY9,34.162491,70.763668,0,Afghanistan,906.564103,1204.102564,1656.307692,2044.384615,2355.641026,2441.410256,2556.256410,2505.871795,2666.333333,2627.179487,3488.948718,3230.794872,293.897436,1220.205128,5.000000,208.641026,168.743590,122.487179,0.013030,0.107129,-0.133644,0.204115,0.155626,-0.104866,-0.356181,0.101412,-0.781621,124.495128,0.512926,0.631574,-0.133644,-0.048571,2.820813,0.017880,1.059189,1.375554,0.158091,452.205128,9.273460e+06,1.867409,-2.363027e+09,-4.069411e+09,0.197263,0.134879,0.000461,0.275558,0.258669,682.425262,0.304694,-1252.967858,450.285366,1495.801929,0.200912,0.101420,9.561882e-08,3.704675e+05,-0.356181,0.028744,0.030902,0.063775,-0.163988,0.775513,0.009774,-0.163988,8298.461538,64.128205,0.344179,6.841348,0.547340,0.384848,1.079904,1.152249,1.085164,1.047041,1.225734
1,ID_CWCD60FGJJYY,32.075695,48.492047,0,Iran,509.023810,628.214286,907.428571,980.476190,1343.833333,2158.309524,2411.404762,2702.309524,2653.714286,3001.452381,2105.380952,1433.095238,207.833333,1250.833333,4.880952,100.023810,92.500000,64.404762,0.111920,0.660156,0.115218,0.497233,0.851305,-0.038692,-0.397620,0.467491,-0.567875,141.832143,1.977986,0.700900,0.115218,0.258826,1.204585,0.232565,1.975643,1.444457,0.181822,279.214286,5.101703e+06,3.217713,-4.557431e+09,-5.568567e+08,-0.038132,0.621767,0.000180,2.241531,0.218974,720.130890,0.718119,-698.251623,731.191567,1903.427628,1.110987,0.467536,1.210172e-07,1.597925e+07,-0.397620,0.327876,0.335746,1.010896,0.124161,0.983634,0.060220,0.124161,67748.095238,206.166667,0.163212,28.372845,0.560952,0.378785,1.469114,1.370593,1.794422,1.117265,2.756119
2,ID_R1XF70RMVGL3,14.542826,33.313483,1,Sudan,396.586957,593.000000,933.173913,1132.826087,1405.130435,2060.065217,2381.217391,2504.565217,2546.304348,2701.065217,2022.173913,1487.847826,183.565217,1495.913043,4.913043,115.565217,95.239130,60.630435,0.097379,0.486910,0.114728,0.457100,0.656416,-0.096637,-0.368485,0.377085,-0.637881,129.641739,1.683921,0.729678,0.114728,0.155115,1.393534,0.189004,1.649387,1.573649,0.222893,340.173913,5.174432e+06,3.258145,-3.888513e+09,-6.245948e+08,0.009186,0.501526,0.000216,2.405157,0.312793,723.805225,0.745667,-833.047393,465.670882,1710.964077,0.706351,0.377122,1.323519e-07,1.041703e+07,-0.368485,0.245108,0.281207,0.782443,0.106565,0.936548,0.039119,0.106565,47648.260870,140.521739,0.262043,22.744506,0.424987,0.282242,1.359127,1.240376,1.694659,1.155894,2.210900
3,ID_0ZBIDY0PEBVO,14.359480,33.284108,1,Sudan,505.400000,707.844444,1068.200000,1391.266667,1743.844444,2139.222222,2342.777778,2402.777778,2512.911111,2447.666667,2196.888889,1715.666667,193.400000,1611.066667,5.044444,141.866667,108.955556,72.333333,0.058026,0.314170,0.067099,0.384496,0.440337,-0.131356,-0.345684,0.266580,-0.705460,93.820889,1.249371,0.712736,0.067099,0.055673,1.675645,0.101821,1.319254,1.509089,0.202898,360.355556,5.909552e+06,2.929671,-3.378955e+09,-1.048740e+09,0.071287,0.354554,0.000279,1.121392,0.325577,715.904058,0.596045,-1251.806619,549.612710,1569.199475,0.464632,0.266605,1.297287e-07,5.721595e+06,-0.345684,0.137654,0.158908,0.377862,0.044762,0.875560,0.008804,0.044762,31954.666667,37.288889,0.319472,16.421751,0.473132,0.289819,1.280487,1.253422,1.343456,1.095154,1.727043
4,ID_C20R2C0AYIT0,14.419128,33.528450,0,Sudan,759.197802,1210.835165,1752.703297,2247.384615,2547.175824,2738.549451,2853.956044,2789.395604,2878.000000,2942.219780,3012.582418,2814.835165,187.989011,1712.098901,5.252747,228.472527,178.439560,124.505495,0.009198,0.114080,-0.022847,0.228241,0.165568,-0.123668,-0.264387,0.107603,-0.779174,137.61384

In [24]:
full_test.head()

,ID,Lat,Lon,location,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12,AOT,WVP,SCL,TCI_R,TCI_G,TCI_B,NDRE,MMSR,NDWI,GNDVI,EVI2,NGRDI,MNDWI,OSAVI,WDRVI,TGI,GCVI,RGVI,MI,ARVI,SIPI,RENDVI,MRESR,RYI,NDYI,DYI,ACI,CVI,AVI,SI,BSI,SAVI,FIDET,MTCI,NPCRI,S2REP,CCCI,MCARI,TCARI,PVI,EVI,NDVI,BAI,MTVI2,NDSI,MRENDVI,NDVIre,CIre,NDMI,TNDVI,VDVI,NBR,TVI,EXG,PSRI,RDVI,RATIO1,RATIO2,RATIO3,RATIO4,RATIO5,RATIO6,RATIO7
0,ID_9ZLHTVF6NSU7,34.254835,70.348699,Afghanistan,631.621622,881.202703,1312.513514,1409.081081,1848.824324,2574.040541,2857.500000,2914.486486,2942.256757,2994.608108,2765.608108,2266.500000,269.567568,1060.783784,4.702703,143.635135,134.040541,90.040541,0.062029,0.438179,0.030948,0.378986,0.597641,-0.035482,-0.356315,0.348158,-0.657226,225.438243,1.220538,0.682092,0.030948,0.147800,1.350655,0.163970,1.595806,1.489457,0.196612,431.310811,7.932051e+06,2.383913,-6.177933e+09,-1.625486e+09,0.047551,0.463052,0.000257,1.649181,0.230486,718.728758,0.642539,-830.385814,897.020752,1969.515670,0.790496,0.348186,9.542796e-08,1.469609e+07,-0.356315,0.229526,0.223723,0.576400,0.026211,0.920970,0.068102,0.026211,66034.864865,334.743243,0.205078,22.894562,0.481231,0.341634,1.220211,1.312078,1.545577,1.110122,2.068360
1,ID_LNN7BFCVEZKA,32.009669,48.535526,Iran,976.666667,1393.642857,1914.380952,2407.476190,2633.666667,2824.809524,2989.738095,3185.166667,3120.904762,3217.047619,3631.547619,3163.190476,207.285714,1300.619048,5.000000,228.333333,194.785714,143.952381,0.059960,0.150231,-0.075623,0.249196,0.216890,-0.114093,-0.309627,0.139047,-0.766311,125.343095,0.663810,0.640009,-0.075623,-0.025138,2.303646,0.035018,1.115355,1.373652,0.157417,520.738095,1.376584e+07,2.092365,-5.961019e+09,-6.412432e+09,0.137525,0.184934,0.000315,0.845053,0.266720,716.891193,0.681585,-1326.288216,206.452414,1946.716951,0.270833,0.139056,6.273382e-08,2.791952e+06,-0.309627,0.054532,0.094778,0.209404,-0.065483,0.799410,0.003623,-0.065483,5316.190476,27.642857,0.358903,10.399159,0.510174,0.370839,1.148065,1.093953,1.135200,1.058386,1.323031
2,ID_SOYSG7W04UH3,14.431884,33.399991,Sudan,451.659341,628.120879,937.439560,1166.978022,1454.527473,1964.351648,2180.395604,2073.318681,2334.780220,2445.857143,1962.461538,1530.043956,188.000000,1602.846154,5.142857,118.890110,95.428571,64.219780,0.026988,0.332913,0.086641,0.377273,0.464784,-0.109075,-0.353468,0.279679,-0.698275,99.164396,1.211683,0.710719,0.086641,0.072980,1.594542,0.149120,1.508366,1.492451,0.197577,309.318681,4.363128e+06,2.753228,-2.191028e+09,-6.847332e+08,0.073402,0.371975,0.000268,1.772997,0.300182,720.045534,0.627087,-930.598993,475.947715,1361.933913,0.519062,0.279709,1.766783e-07,4.908277e+06,-0.353468,0.202668,0.175402,0.425424,0.027469,0.883011,0.021739,0.027469,38660.879121,79.780220,0.274318,15.922053,0.481801,0.310520,1.282618,1.246405,1.499041,1.109982,1.776656
3,ID_EAP7EXXV8ZDE,14.281866,33.441224,Sudan,379.888889,539.533333,824.155556,998.155556,1274.666667,1800.400000,1966.466667,2065.533333,2095.911111,2263.222222,1756.155556,1373.888889,193.111111,1686.755556,4.177778,101.800000,84.355556,55.511111,0.068582,0.438524,0.088201,0.429589,0.598023,-0.095483,-0.361197,0.348356,-0.657090,105.759556,1.506242,0.715980,0.088201,0.132299,1.429672,0.170966,1.587557,1.527534,0.208715,284.622222,3.764044e+06,3.035373,-2.198433e+09,-4.420358e+08,0.027847,0.463317,0.000290,1.901310,0.298254,718.823654,0.679606,-797.514467,484.345756,1395.818818,0.665619,0.348396,1.900312e-07,6.534504e+06,-0.361197,0.227070,0.236772,0.620450,0.080953,0.921084,0.034721,0.080953,41174.666667,110.622222,0.254734,19.283942,0.460943,0.298030,1.278237,1.277022,1.542730,1.092239,2.069350
4,ID_QPRX1TUQVGHU,14.399365,33.109566,Sudan,789.461538,908.241758,1253.637363,1693.560440,2070.725275,2207.912088,2364.967033,2356.076923,2518.846154,2569.318681,3010.703297,2677.230769,188.131868,1591.318681,5.043956,169.197802,124.461538,88.571429,0.032464,0.179490,-0.088951,0.305409,0.257924,-0.149268,-0.412037,0.163585,-0.755742,39.121319,0.879393,0.666664,-

In [25]:
full_ts_train.head()

,ID,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12,AOT,WVP,SCL,TCI_R,TCI_G,TCI_B,NDRE,MMSR,NDWI,GNDVI,EVI2,NGRDI,MNDWI,OSAVI,WDRVI,TGI,GCVI,RGVI,MI,ARVI,SIPI,RENDVI,MRESR,RYI,NDYI,DYI,ACI,CVI,AVI,SI,BSI,SAVI,FIDET,MTCI,NPCRI,S2REP,CCCI,MCARI,TCARI,PVI,EVI,NDVI,BAI,MTVI2,NDSI,MRENDVI,NDVIre,CIre,NDMI,TNDVI,VDVI,NBR,TVI,EXG,PSRI,RDVI,RATIO1,RATIO2,RATIO3,RATIO4,RATIO5,RATIO6,RATIO7
0,ID_SJ098E7S2SY9,739.0,1058.0,1490.0,1648.0,2005.0,2591.0,2750.0,2708.0,2848.0,2781.0,3153.0,2805.0,214.0,506.0,5.0,168.0,152.0,110.0,0.022080,0.281875,-0.050825,0.290138,0.397647,-0.050351,-0.358174,0.243323,-0.717740,201.90,0.817450,0.651882,-0.050825,0.066553,1.556604,0.127502,1.462875,1.408318,0.169545,432.0,8497704.0,2.010173,-4.727681e+09,-2.592169e+09,0.120812,0.323621,0.000354,1.641457,0.218034,716.587031,0.612971,-818.789442,695.062500,1749.609172,0.568426,0.243343,9.951721e-08,8.664288e+06,-0.358174,0.187941,0.149162,0.350623,-0.075926,0.862173,0.048189,-0.075926,50260.0,274.0,0.227711,16.060606,0.495973,0.368579,1.124064,1.216626,1.371571,1.061366,1.643204
1,ID_SJ098E7S2SY9,778.0,966.0,1380.0,1574.0,1967.0,2487.0,2599.0,2644.0,2724.0,2689.0,3067.0,2651.0,246.0,664.0,5.0,159.0,141.0,101.0,0.030598,0.296070,-0.059230,0.314115,0.416498,-0.065674,-0.379357,0.253654,-0.712359,176.88,0.915942,0.648534,-0.059230,0.068363,1.568224,0.116749,1.437342,1.428571,0.176471,414.0,7810376.0,2.185285,-4.450143e+09,-2.093246e+09,0.124955,0.337361,0.000362,1.323155,0.239370,713.043269,0.578784,-976.001906,738.861881,1716.563371,0.552230,0.253675,1.056236e-07,8.183493e+06,-0.379357,0.179434,0.146823,0.344179,-0.074068,0.868144,0.041509,-0.074068,47020.0,220.0,0.244471,16.475192,0.563768,0.395526,1.156922,1.249682,1.321301,1.045034,1.679797
2,ID_SJ098E7S2SY9,800.0,911.0,1436.0,1868.0,2210.0,2369.0,2524.0,2512.0,2725.0,2699.0,3362.0,2861.0,378.0,1481.0,5.0,190.0,146.0,92.0,0.029297,0.159635,-0.104649,0.272543,0.230125,-0.130751,-0.401417,0.147020,-0.762929,151.77,0.749304,0.661315,-0.104649,-0.043721,2.486025,0.034724,1.112766,1.576290,0.223690,525.0,8299648.0,2.275557,-3.020298e+09,-2.438022e+09,0.208829,0.195538,0.000389,0.464912,0.344368,701.918239,0.434980,-1426.798715,476.576017,1542.479986,0.233723,0.147032,1.020523e-07,1.591452e+06,-0.401417,0.053374,0.063956,0.136652,-0.144705,0.804383,0.016457,-0.144705,12780.0,93.0,0.403968,9.730806,0.557103,0.361991,1.175114,1.183084,1.142081,1.065428,1.344754
3,ID_SJ098E7S2SY9,564.0,999.0,1500.0,1930.0,2278.0,2370.0,2571.0,2492.0,2740.0,2714.0,3527.0,3098.0,562.0,1903.0,5.0,199.0,154.0,101.0,0.025093,0.136306,-0.125578,0.248497,0.197193,-0.125364,-0.403223,0.127082,-0.771292,137.91,0.661333,0.695530,-0.125578,-0.050197,2.656584,0.019793,1.053676,1.501502,0.200480,501.0,8547560.0,2.137582,-2.701572e+09,-2.885788e+09,0.219714,0.169019,0.000417,0.264368,0.317856,694.538043,0.353003,-1425.815544,493.030881,1511.964006,0.213510,0.127092,1.006616e-07,8.626477e+05,-0.403223,0.026136,0.044864,0.093942,-0.171955,0.791891,0.011975,-0.171955,9200.0,71.0,0.392827,8.451367,0.376000,0.247586,1.138476,1.180311,1.128622,1.084810,1.291192
4,ID_SJ098E7S2SY9,900.0,1040.0,1454.0,1932.0,2155.0,2272.0,2392.0,2366.0,2548.0,2554.0,3370.0,2935.0,246.0,1522.0,5.0,198.0,148.0,105.0,0.020267,0.106633,-0.138898,0.238743,0.154916,-0.141170,-0.397181,0.100969,-0.781795,66.12,0.627235,0.636672,-0.138898,-0.062999,3.055300,0.026429,1.093227,1.398077,0.165998,414.0,8011276.0,2.162186,-1.982836e+09,-2.915167e+09,0.217731,0.134289,0.000451,0.524664,0.300135,707.094017,0.462194,-1315.085404,199.852484,1411.877918,0.176165,0.100977,1.071822e-07,-7.139869e+05,-0.397181,0.044537,0.046671,0.097912,-0.175035,0.775227,-0.010884,-0.175035,1280.0,-64.0,0.392606,6.619978,0.618982,0.417633,1.148211,1.115424,1.109977,1.052817,1.224638


In [26]:
full_ts_test.head()

,ID,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12,AOT,WVP,SCL,TCI_R,TCI_G,TCI_B,NDRE,MMSR,NDWI,GNDVI,EVI2,NGRDI,MNDWI,OSAVI,WDRVI,TGI,GCVI,RGVI,MI,ARVI,SIPI,RENDVI,MRESR,RYI,NDYI,DYI,ACI,CVI,AVI,SI,BSI,SAVI,FIDET,MTCI,NPCRI,S2REP,CCCI,MCARI,TCARI,PVI,EVI,NDVI,BAI,MTVI2,NDSI,MRENDVI,NDVIre,CIre,NDMI,TNDVI,VDVI,NBR,TVI,EXG,PSRI,RDVI,RATIO1,RATIO2,RATIO3,RATIO4,RATIO5,RATIO6,RATIO7
0,ID_9ZLHTVF6NSU7,421.0,307.0,606.0,466.0,950.0,2324.0,3006.0,3880.0,2926.0,3230.0,1589.0,1017.0,204.0,504.0,4.0,49.0,63.0,31.0,0.250806,1.885512,0.296124,0.729826,1.707205,0.130597,-0.447836,0.785487,-0.091335,236.99,5.402640,0.767752,0.296124,0.635866,1.046573,0.419670,3.597353,1.973941,0.327492,299.0,4159360.0,4.923482,-6.159539e+09,-86085450.0,-0.341557,1.044702,0.000108,2.838843,0.205692,725.021834,0.772230,-415.879828,1031.227468,2956.492441,1.951080,0.785550,6.548361e-08,4.980207e+07,-0.447836,0.564967,0.606625,3.084211,0.418907,1.133821,0.221159,0.418907,117080.0,439.0,0.068417,51.786750,0.694719,0.443158,1.562439,2.038627,3.164211,1.293460,8.326180
1,ID_9ZLHTVF6NSU7,425.0,319.0,616.0,478.0,960.0,2324.0,3000.0,3872.0,2922.0,3256.0,1592.0,1022.0,189.0,503.0,4.0,49.0,64.0,33.0,0.249839,1.846123,0.294639,0.725490,1.690172,0.126143,-0.442029,0.780167,-0.104947,234.99,5.285714,0.765435,0.294639,0.628521,1.046847,0.415347,3.549533,1.931034,0.317647,297.0,4235968.0,4.877551,-6.268528e+09,-93286890.0,-0.338764,1.037627,0.000107,2.829876,0.199498,724.989003,0.772399,-413.723849,1031.472803,2947.321836,1.951248,0.780230,6.570169e-08,4.939041e+07,-0.442029,0.560394,0.602649,3.033333,0.417277,1.131472,0.214391,0.417277,116280.0,435.0,0.068417,51.459695,0.689935,0.442708,1.557730,2.008368,3.125000,1.290878,8.100418
2,ID_9ZLHTVF6NSU7,428.0,429.0,755.0,577.0,1204.0,2470.0,2940.0,3286.0,2885.0,3116.0,1875.0,1171.0,234.0,651.0,4.0,58.0,77.0,44.0,0.141765,1.386414,0.212185,0.626330,1.449655,0.133634,-0.425856,0.701205,-0.274293,268.28,3.352318,0.749417,0.212185,0.525981,1.054633,0.344584,2.631443,1.759907,0.275338,326.0,4376952.0,3.326208,-5.127422e+09,-185882112.0,-0.204800,0.932607,0.000130,2.019139,0.147117,720.329779,0.661226,-565.483536,1318.855113,2460.568247,1.917740,0.701268,8.984552e-08,3.432496e+07,-0.425856,0.449255,0.463697,1.729236,0.273397,1.096024,0.200318,0.273397,120700.0,504.0,0.059919,43.585964,0.566887,0.355482,1.601196,2.086655,2.441860,1.190283,5.694974
3,ID_9ZLHTVF6NSU7,415.0,418.0,745.0,569.0,1195.0,2464.0,2934.0,3274.0,2882.0,3141.0,1871.0,1170.0,229.0,653.0,4.0,57.0,77.0,43.0,0.141164,1.398740,0.212708,0.629261,1.457247,0.133942,-0.430428,0.703813,-0.269523,268.11,3.394631,0.753086,0.212708,0.528778,1.055823,0.346816,2.626923,1.782297,0.281169,327.0,4302036.0,3.356436,-5.030305e+09,-176220864.0,-0.204175,0.936076,0.000129,2.027157,0.152989,720.348700,0.660917,-575.448155,1310.952548,2452.978781,1.902786,0.703877,9.056059e-08,3.412693e+07,-0.430428,0.448568,0.465205,1.739749,0.272692,1.097213,0.203068,0.272692,120740.0,503.0,0.061282,43.634708,0.557047,0.347280,1.599145,2.100176,2.455230,1.190747,5.753954
4,ID_9ZLHTVF6NSU7,428.0,443.0,857.0,725.0,1377.0,3588.0,4192.0,4086.0,4324.0,4239.0,2246.0,1325.0,359.0,1667.0,4.0,76.0,87.0,45.0,0.064894,1.373997,0.316286,0.653247,1.441994,0.083439,-0.447631,0.698557,-0.279111,304.02,3.767795,0.795837,0.316286,0.514969,1.083904,0.445317,3.329821,1.934537,0.318462,414.0,6464052.0,4.033432,-9.942725e+09,-273926848.0,-0.207733,0.929084,0.000072,3.391104,0.241438,722.120081,0.709814,-736.932414,1363.415172,3057.922497,1.642878,0.698607,5.807071e-08,5.153934e+07,-0.447631,0.538087,0.495881,1.967320,0.290587,1.094809,0.189452,0.290587,177060.0,546.0,0.078595,48.456365,0.499417,0.310821,1.695094,1.899310,3.044299,1.168339,5.635862


In [27]:
full_train_agg = agg_data(full_ts_train)
full_train_agg = pd.merge(full_train_agg, train[['ID', 'Lat', 'Lon', 'Target', 'location']], on='ID')

full_test_agg = agg_data(full_ts_test)
full_test_agg = pd.merge(full_test_agg, test[['ID', 'Lat', 'Lon', 'location']], on='ID')

In [28]:
full_ts_train_aug = pd.merge(full_ts_train, train[['ID', 'Lat', 'Lon', 'Target', 'location']], on='ID')
full_ts_test_aug = pd.merge(full_ts_test, test[['ID', 'Lat', 'Lon', 'location']], on='ID')

In [29]:
full_train_agg.head()

,ID,B1_mean,B2_mean,B3_mean,B4_mean,B5_mean,B6_mean,B7_mean,B8_mean,B8A_mean,B9_mean,B11_mean,B12_mean,AOT_mean,WVP_mean,SCL_mean,TCI_R_mean,TCI_G_mean,TCI_B_mean,NDRE_mean,MMSR_mean,NDWI_mean,GNDVI_mean,EVI2_mean,NGRDI_mean,MNDWI_mean,OSAVI_mean,WDRVI_mean,TGI_mean,GCVI_mean,RGVI_mean,MI_mean,ARVI_mean,SIPI_mean,RENDVI_mean,MRESR_mean,RYI_mean,NDYI_mean,DYI_mean,ACI_mean,CVI_mean,AVI_mean,SI_mean,BSI_mean,SAVI_mean,FIDET_mean,MTCI_mean,NPCRI_mean,S2REP_mean,CCCI_mean,MCARI_mean,TCARI_mean,PVI_mean,EVI_mean,NDVI_mean,BAI_mean,MTVI2_mean,NDSI_mean,MRENDVI_mean,NDVIre_mean,CIre_mean,NDMI_mean,TNDVI_mean,VDVI_mean,NBR_mean,TVI_mean,EXG_mean,PSRI_mean,RDVI_mean,RATIO1_mean,RATIO2_mean,RATIO3_mean,RATIO4_mean,RATIO5_mean,RATIO6_mean,RATIO7_mean,B1_min,B2_min,B3_min,B4_min,B5_min,B6_min,B7_min,B8_min,B8A_min,B9_min,B11_min,B12_min,AOT_min,WVP_min,SCL_min,TCI_R_min,TCI_G_min,TCI_B_min,NDRE_min,MMSR_min,NDWI_min,GNDVI_min,EVI2_min,NGRDI_min,MNDWI_min,OSAVI_min,WDRVI_min,TGI_min,GCVI_min,RGVI_min,MI_min,ARVI_min,SIPI_min,RENDVI_min,MRESR_min,RYI_min,NDYI_min,DYI_min,ACI_min,CVI_min,AVI_min,SI_min,BSI_min,SAVI_min,FIDET_min,MTCI_min,NPCRI_min,S2REP_min,CCCI_min,MCARI_min,TCARI_min,PVI_min,EVI_min,NDVI_min,BAI_min,MTVI2_min,NDSI_min,MRENDVI_min,NDVIre_min,CIre_min,NDMI_min,TNDVI_min,VDVI_min,NBR_min,TVI_min,EXG_min,PSRI_min,RDVI_min,RATIO1_min,RATIO2_min,RATIO3_min,RATIO4_min,RATIO5_min,RATIO6_min,RATIO7_min,B1_max,B2_max,B3_max,B4_max,B5_max,B6_max,B7_max,B8_max,B8A_max,B9_max,B11_max,B12_max,AOT_max,WVP_max,SCL_max,TCI_R_max,TCI_G_max,TCI_B_max,NDRE_max,MMSR_max,NDWI_max,GNDVI_max,EVI2_max,NGRDI_max,MNDWI_max,OSAVI_max,WDRVI_max,TGI_max,GCVI_max,RGVI_max,MI_max,ARVI_max,SIPI_max,RENDVI_max,MRESR_max,RYI_max,NDYI_max,DYI_max,ACI_max,CVI_max,AVI_max,SI_max,BSI_max,SAVI_max,FIDET_max,MTCI_max,NPCRI_max,S2REP_max,CCCI_max,MCARI_max,TCARI_max,PVI_max,EVI_max,NDVI_max,BAI_max,MTVI2_max,NDSI_max,MRENDVI_max,NDVIre_max,CIre_max,NDMI_max,TNDVI_max,VDVI_max,NBR_max,TVI_max,EXG_max,PSRI_max,RDVI_max,RATIO1_max,RATIO2_max,RATIO3_max,RATIO4_max,RATIO5_max,RATIO6_max,RATIO7_max,Lat,Lon,Target,location
0,ID_00MJRGGCU6WE,741.641026,1030.333333,1464.205128,1810.205128,2198.846154,2451.871795,2652.282051,2525.102564,2762.153846,2833.564103,3212.435897,2856.538462,289.435897,1234.153846,4.974359,184.179487,149.230769,105.692308,0.014618,0.195278,-0.075685,0.267284,0.271911,-0.104523,-0.375446,0.167064,-0.748692,129.721795,0.774171,0.670066,-0.075685,-0.003998,2.355447,0.054861,1.179934,1.433379,0.176316,433.871795,8.378059e+06,2.180106,-3.278257e+09,-2.971079e+09,0.168977,0.222196,0.000392,0.000132,0.275559,697.339247,0.332456,-1342.956381,612.162057,1566.545730,0.334876,0.167078,1.103433e-07,3.384198e+06,-0.375446,0.073823,0.069170,0.166929,-0.118815,0.814859,0.016393,-0.118815,24660.000000,87.871795,0.320678,10.900037,0.508420,0.333857,1.132812,1.235225,1.223618,1.081164,1.452941,237.0,414.0,726.0,710.0,1299.0,1322.0,1441.0,1475.0,1466.0,1423.0,1961.0,1777.0,112.0,310.0,4.0,70.0,74.0,47.0,-0.096328,0.099054,-0.169738,0.200998,0.144053,-0.153341,-0.521669,0.094163,-0.784452,23.10,0.503121,0.611136,-0.169738,-0.068478,1.145455,-0.013915,0.955090,1.248344,0.110457,242.0,2983925.0,1.736957,-7.486925e+09,-5.390681e+09,-0.024485,0.125237,0.000192,-25.500000,0.186474,-240.000000,-0.957182,-2085.548293,-181.033200,895.082692,0.166741,0.094170,6.882529e-08,-5.916269e+05,-0.521669,-0.022971,-0.099920,-0.181687,-0.298492,0.770824,-0.021209,-0.298492,-7720.0,-134.0,0.123746,5.788719,0.259279,0.129367,0.979629,0.983000,0.990889,1.009280,1.207921,1003.0,1288.0,1849.0,2344.0,2739.0,2940.0,3261.0,3176.0,3343.0,3293.0,3804.0,3376.0,553.0,3541.0,5.0,236.0,187.0,133.0,0.084681,0.966265,0.124738,0.581677,1.143258,0.011142,-0.236508,0.588942,-0.442357,326.22,2.780992,0.804764,0.124738,0.379777,3.170404,0.245014,1.904470,1.850932,0.298475,618.0,12604158.0,4.049998,-5.616210e+08,-212292325.0,0.295129,0.783297,0.000861,3.634146,0.428064,1463.333333,1.037245,-296.866000,1355.7

In [30]:
full_test_agg.head()

,ID,B1_mean,B2_mean,B3_mean,B4_mean,B5_mean,B6_mean,B7_mean,B8_mean,B8A_mean,B9_mean,B11_mean,B12_mean,AOT_mean,WVP_mean,SCL_mean,TCI_R_mean,TCI_G_mean,TCI_B_mean,NDRE_mean,MMSR_mean,NDWI_mean,GNDVI_mean,EVI2_mean,NGRDI_mean,MNDWI_mean,OSAVI_mean,WDRVI_mean,TGI_mean,GCVI_mean,RGVI_mean,MI_mean,ARVI_mean,SIPI_mean,RENDVI_mean,MRESR_mean,RYI_mean,NDYI_mean,DYI_mean,ACI_mean,CVI_mean,AVI_mean,SI_mean,BSI_mean,SAVI_mean,FIDET_mean,MTCI_mean,NPCRI_mean,S2REP_mean,CCCI_mean,MCARI_mean,TCARI_mean,PVI_mean,EVI_mean,NDVI_mean,BAI_mean,MTVI2_mean,NDSI_mean,MRENDVI_mean,NDVIre_mean,CIre_mean,NDMI_mean,TNDVI_mean,VDVI_mean,NBR_mean,TVI_mean,EXG_mean,PSRI_mean,RDVI_mean,RATIO1_mean,RATIO2_mean,RATIO3_mean,RATIO4_mean,RATIO5_mean,RATIO6_mean,RATIO7_mean,B1_min,B2_min,B3_min,B4_min,B5_min,B6_min,B7_min,B8_min,B8A_min,B9_min,B11_min,B12_min,AOT_min,WVP_min,SCL_min,TCI_R_min,TCI_G_min,TCI_B_min,NDRE_min,MMSR_min,NDWI_min,GNDVI_min,EVI2_min,NGRDI_min,MNDWI_min,OSAVI_min,WDRVI_min,TGI_min,GCVI_min,RGVI_min,MI_min,ARVI_min,SIPI_min,RENDVI_min,MRESR_min,RYI_min,NDYI_min,DYI_min,ACI_min,CVI_min,AVI_min,SI_min,BSI_min,SAVI_min,FIDET_min,MTCI_min,NPCRI_min,S2REP_min,CCCI_min,MCARI_min,TCARI_min,PVI_min,EVI_min,NDVI_min,BAI_min,MTVI2_min,NDSI_min,MRENDVI_min,NDVIre_min,CIre_min,NDMI_min,TNDVI_min,VDVI_min,NBR_min,TVI_min,EXG_min,PSRI_min,RDVI_min,RATIO1_min,RATIO2_min,RATIO3_min,RATIO4_min,RATIO5_min,RATIO6_min,RATIO7_min,B1_max,B2_max,B3_max,B4_max,B5_max,B6_max,B7_max,B8_max,B8A_max,B9_max,B11_max,B12_max,AOT_max,WVP_max,SCL_max,TCI_R_max,TCI_G_max,TCI_B_max,NDRE_max,MMSR_max,NDWI_max,GNDVI_max,EVI2_max,NGRDI_max,MNDWI_max,OSAVI_max,WDRVI_max,TGI_max,GCVI_max,RGVI_max,MI_max,ARVI_max,SIPI_max,RENDVI_max,MRESR_max,RYI_max,NDYI_max,DYI_max,ACI_max,CVI_max,AVI_max,SI_max,BSI_max,SAVI_max,FIDET_max,MTCI_max,NPCRI_max,S2REP_max,CCCI_max,MCARI_max,TCARI_max,PVI_max,EVI_max,NDVI_max,BAI_max,MTVI2_max,NDSI_max,MRENDVI_max,NDVIre_max,CIre_max,NDMI_max,TNDVI_max,VDVI_max,NBR_max,TVI_max,EXG_max,PSRI_max,RDVI_max,RATIO1_max,RATIO2_max,RATIO3_max,RATIO4_max,RATIO5_max,RATIO6_max,RATIO7_max,Lat,Lon,location
0,ID_00SYEM165IJF,1216.608108,2022.837838,2621.554054,2967.770270,2702.324324,2799.702703,2896.175676,3249.635135,2973.864865,2852.581081,3527.905405,3253.148649,270.405405,1013.716216,5.081081,247.851351,237.729730,203.891892,0.078155,0.050265,-0.087510,0.111334,0.073383,-0.063159,-0.151505,0.048579,-0.801223,230.192568,0.253542,0.552899,-0.087510,-0.058472,4.687864,0.017935,1.070837,1.308157,0.132722,598.716216,1.865662e+07,1.424838,-2.576181e+09,-1.758309e+10,0.105211,0.064610,0.000405,-0.423939,0.194168,792.706885,2.133499,-396.853726,-846.230372,1864.798341,0.122937,0.048582,5.655125e-08,-1.235649e+06,-0.151505,0.032181,0.095874,0.217628,-0.040922,0.740432,0.025304,-0.040922,-23932.702703,252.500000,0.345255,3.686029,0.464644,0.449001,1.084703,0.911364,1.074722,1.036567,1.103907,342.0,840.0,1434.0,1654.0,1489.0,1474.0,1815.0,2142.0,1740.0,1630.0,2241.0,1926.0,93.0,202.0,5.0,167.0,146.0,86.0,0.028227,-0.010215,-0.157198,0.041388,-0.015034,-0.109471,-0.225493,-0.010267,-0.821548,97.90,0.086349,0.520346,-0.157198,-0.087993,-22.681818,-0.038766,0.870021,1.217755,0.098187,440.0,6670080.0,1.204641,-5.397241e+09,-3.602461e+10,0.068292,-0.013655,0.000276,-14.000000,0.147301,-124.500000,-6.690744,-841.360756,-2320.782845,1294.794588,-0.028975,-0.010268,3.209648e-08,-6.397591e+06,-0.225493,-0.069507,0.036468,0.075697,-0.096978,0.699809,-0.001031,-0.096978,-45760.0,-12.0,0.265996,-0.853812,0.215447,0.221360,1.021017,0.697792,0.980837,0.921364,0.979674,1612.0,2796.0,3450.0,3858.0,3494.0,3525.0,3758.0,4034.0,3703.0,3591.0,4288.0,3981.0,534.0,3018.0,7.0,255.0,255.0,255.0,0.188773,0.142771,-0.038777,0.202003,0.206342,-0.033094,-0.070751,0.132657,-0.768984,372.07,0.506276,0.641791,-0.038777,-0.011874,16.028571,0.083868,1.345064,1.707143,0.261214,792.0,29480472.0,1.804771,8.484233e+08,-1.987380e+09,0.153736,0.176435,0.000758,12.100000,0.326383,2980.000000,5.847797,31.400

In [31]:
df_train = full_train_agg.copy()
test_df = full_test_agg.copy()

In [34]:
n_split = 5
df_train['folds'] = -1

split_target = df_train['location'] + '_' + df_train['Target'].astype(str)

skf = StratifiedKFold(n_split, shuffle=True, random_state=SEED)

for i, (tr, vr) in enumerate(skf.split(df_train, split_target)):
    df_train.loc[vr, 'folds'] = i

In [35]:
target = 'Target'
columns = df_train.columns.difference(['ID', 'location', 'folds', 'label', target])
columns

Index(['ACI_max', 'ACI_mean', 'ACI_min', 'AOT_max', 'AOT_mean', 'AOT_min',
       'ARVI_max', 'ARVI_mean', 'ARVI_min', 'AVI_max',
       ...
       'TVI_min', 'VDVI_max', 'VDVI_mean', 'VDVI_min', 'WDRVI_max',
       'WDRVI_mean', 'WDRVI_min', 'WVP_max', 'WVP_mean', 'WVP_min'],
      dtype='object', length=227)

In [36]:
df_train.describe()

,B1_mean,B2_mean,B3_mean,B4_mean,B5_mean,B6_mean,B7_mean,B8_mean,B8A_mean,B9_mean,B11_mean,B12_mean,AOT_mean,WVP_mean,SCL_mean,TCI_R_mean,TCI_G_mean,TCI_B_mean,NDRE_mean,MMSR_mean,NDWI_mean,GNDVI_mean,EVI2_mean,NGRDI_mean,MNDWI_mean,OSAVI_mean,WDRVI_mean,TGI_mean,GCVI_mean,RGVI_mean,MI_mean,ARVI_mean,SIPI_mean,RENDVI_mean,MRESR_mean,RYI_mean,NDYI_mean,DYI_mean,ACI_mean,CVI_mean,AVI_mean,SI_mean,BSI_mean,SAVI_mean,FIDET_mean,MTCI_mean,NPCRI_mean,S2REP_mean,CCCI_mean,MCARI_mean,TCARI_mean,PVI_mean,EVI_mean,NDVI_mean,BAI_mean,MTVI2_mean,NDSI_mean,MRENDVI_mean,NDVIre_mean,CIre_mean,NDMI_mean,TNDVI_mean,VDVI_mean,NBR_mean,TVI_mean,EXG_mean,PSRI_mean,RDVI_mean,RATIO1_mean,RATIO2_mean,RATIO3_mean,RATIO4_mean,RATIO5_mean,RATIO6_mean,RATIO7_mean,B1_min,B2_min,B3_min,B4_min,B5_min,B6_min,B7_min,B8_min,B8A_min,B9_min,B11_min,B12_min,AOT_min,WVP_min,SCL_min,TCI_R_min,TCI_G_min,TCI_B_min,NDRE_min,MMSR_min,NDWI_min,GNDVI_min,EVI2_min,NGRDI_min,MNDWI_min,OSAVI_min,WDRVI_min,TGI_min,GCVI_min,RGVI_min,MI_min,ARVI_min,SIPI_min,RENDVI_min,MRESR_min,RYI_min,NDYI_min,DYI_min,ACI_min,CVI_min,AVI_min,SI_min,BSI_min,SAVI_min,FIDET_min,MTCI_min,NPCRI_min,S2REP_min,CCCI_min,MCARI_min,TCARI_min,PVI_min,EVI_min,NDVI_min,BAI_min,MTVI2_min,NDSI_min,MRENDVI_min,NDVIre_min,CIre_min,NDMI_min,TNDVI_min,VDVI_min,NBR_min,TVI_min,EXG_min,PSRI_min,RDVI_min,RATIO1_min,RATIO2_min,RATIO3_min,RATIO4_min,RATIO5_min,RATIO6_min,RATIO7_min,B1_max,B2_max,B3_max,B4_max,B5_max,B6_max,B7_max,B8_max,B8A_max,B9_max,B11_max,B12_max,AOT_max,WVP_max,SCL_max,TCI_R_max,TCI_G_max,TCI_B_max,NDRE_max,MMSR_max,NDWI_max,GNDVI_max,EVI2_max,NGRDI_max,MNDWI_max,OSAVI_max,WDRVI_max,TGI_max,GCVI_max,RGVI_max,MI_max,ARVI_max,SIPI_max,RENDVI_max,MRESR_max,RYI_max,NDYI_max,DYI_max,ACI_max,CVI_max,AVI_max,SI_max,BSI_max,SAVI_max,FIDET_max,MTCI_max,NPCRI_max,S2REP_max,CCCI_max,MCARI_max,TCARI_max,PVI_max,EVI_max,NDVI_max,BAI_max,MTVI2_max,NDSI_max,MRENDVI_max,NDVIre_max,CIre_max,NDMI_max,TNDVI_max,VDVI_max,NBR_max,TVI_max,EXG_max,PSRI_max,RDVI_max,RATIO1_max,RATIO2_max,RATIO3_max,RATIO4_max,RATIO5_max,RATIO6_max,RATIO7_max,Lat,Lon,Target,folds
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1496.000000,1500.000000,1499.000000,1488.000000,1500.000000,1500.000000,1.500000e+03,1500.000000,1.500000e+03,1.500000e+03,1500.000000,1500.000000,1500.000000,1472.000000,1500.000000,1391.000000,1496.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1.500000e+03,1.500000e+03,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1.500000e+03,1500.000000,1.500000e+03,1.500000e+03,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1.500000e+03,1.500000e+03,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.

In [39]:
def train_k_fold(df, df_test, params, columns, verbose=True):
    oofs = {
        'train': {
            'pred': [],
            'gt' : []
        },
        'test': []
    }
    
    score = 0
    n_folds = df['folds'].nunique()

    for fold in sorted(df['folds'].unique()):
        if verbose:
            print('*'*35)
            print('Fold', fold)

        real = df[df['folds']!=fold].reset_index(drop=True)
        val = df[df['folds']==fold].reset_index(drop=True)

        model = catb.CatBoostClassifier(**params)
        model.fit(
            real[columns], real[target],
            eval_set=[(val[columns], val[target])],
            early_stopping_rounds=100,
            verbose = 200 if verbose else 0
        )

        pred = model.predict(val[columns])
        sc = accuracy_score(val[target], pred)

        oofs['train']['gt'].extend(val[target].values)
        oofs['train']['pred'].extend(model.predict_proba(val[columns])[:, 1])
        oofs['test'].append(model.predict_proba(df_test[columns])[:, 1])

        score += sc / n_folds

        if verbose:
            print("Gt Stats:", np.mean(val[target]))
            print("Pr Stats:", np.mean(pred))
            print("Acc score:", sc)

            print('*'*35)
            print()
    if verbose:
        print(f'Avg Acc score: {score:.7f}')
        
    oof_score = accuracy_score(np.array(oofs['train']['gt']), (np.array(oofs['train']['pred']) >= 0.5).astype(int))

    return oof_score, score, oofs

In [58]:
score = 0
all_ids, all_values, oofs = [], [], []
region_test_oofs = {}

params = dict(n_estimators=10000, learning_rate=0.01, colsample_bylevel=0.5)

for region in time_maps.keys():
    region_train_data = df_train[df_train.location==region]
    region_test_data = test_df[test_df.location==region]
    
    region_oof_score, region_score, region_oofs = train_k_fold(
        region_train_data, 
        region_test_data, 
        params, 
        [x for x in columns.values if 'min' in x],
        False
    )
    
    print(f'{region.capitalize()} Accuracy: {region_score:.3f} - Oof: {region_oof_score:.3f}')
    
    oofs.append(region_oofs)
    all_ids.extend(region_test_data.ID.values)
    all_values.extend(np.mean(region_oofs['test'], axis=0))
    
    region_test_oofs[region] = {
        'ids': region_test_data.ID.values, 
        'oof': np.mean(region_oofs['test'], axis=0)
    }
    
    score += region_oof_score/len(time_maps)
    
score = accuracy_score(
    np.array([x for oof in oofs for x in oof['train']['gt']]),
    (np.array([x for oof in oofs for x in oof['train']['pred']]) >= 0.5).astype(int)
)
print()
print(f'OOF Accuracy: {score:.3f}')

Afghanistan Accuracy: 0.876 - Oof: 0.876
Iran Accuracy: 0.956 - Oof: 0.956
Sudan Accuracy: 0.974 - Oof: 0.974

OOF Accuracy: 0.935


In [68]:
sub_file = pd.DataFrame({
    'ID': all_ids,
    'Target': (np.array(all_values) >= 0.5).astype(int)
})
sub_file.head()

,ID,Target
0,ID_00SYEM165IJF,0
1,ID_03A6D3A1NRRW,1
2,ID_03KB5ZE6R3G3,1
3,ID_04IV9TJ35QCD,1
4,ID_04VCSTT3H4LD,0


In [70]:
sub_file.Target.value_counts()

1    752
0    748
Name: Target, dtype: int64

In [71]:
!mkdir -p submissions

In [72]:
# Create file and upload to Zindi for scoring
sub_file.to_csv(f'submissions/v0_catboost_1v1_oof_region_minfeats_{score:.5f}.csv', index = False)